In [1]:

import numpy as np 
import pandas as pd
from nilearn.datasets import fetch_haxby
from nilearn.input_data import NiftiMasker

from reciprocal_isomap import ReciprocalIsomap

from kmapper import KeplerMapper
from landmark_cover import LandmarkCover
from sklearn.cluster import AgglomerativeClustering


/home/poetz/anaconda3/envs/tda/lib/python3.9/site-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


In [2]:

# Fetch dataset, extract time-series from ventral temporal (VT) mask
dataset = fetch_haxby(subjects=[2])
masker = NiftiMasker(
    dataset.mask_vt[0], 
    standardize=True, detrend=True, smoothing_fwhm=4.0,
    low_pass=0.09, high_pass=0.008, t_r=2.5,
    memory="nilearn_cache")
X = masker.fit_transform(dataset.func[0])

# Encode labels as integers
df = pd.read_csv(dataset.session_target[0], sep=" ")
target, labels = pd.factorize(df.labels.values)
timing = pd.DataFrame().assign(task=target, task_name=labels[target])
timing_onehot = pd.DataFrame({l:1*(target==i) for i,l in enumerate(labels)})

In [3]:
# compute distances with reciprocal isomap


# compute geodesic distances on a reciprocal neighbor graph
r_isomap = ReciprocalIsomap(n_neighbors=30, neighbors_mode='distance')
geodesic_distances = r_isomap._reciprocal_distances(X).toarray()


/home/poetz/anaconda3/envs/tda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function graph_shortest_path is deprecated; `graph_shortest_path` is deprecated in 1.0 (renaming of 0.25) and will be removed in 1.2. Use `scipy.sparse.csgraph.shortest_path` instead.
  warnings.warn(msg, category=FutureWarning)


In [5]:
1452/4

363.0

In [10]:
#feed distances to KeplerMapper using Landmark Cover



# setup KeplerMapper object
mapper = KeplerMapper(verbose=1)

# run KeplerMapper using the LandmarkCover
graph = mapper.map(
    lens=geodesic_distances, X=X,
    cover=LandmarkCover(n_landmarks=363, perc_overlap=0.2, metric='precomputed'),
    clusterer=AgglomerativeClustering(n_clusters=10, linkage='single'),
    remove_duplicate_nodes=True,
)

# visualize the graph
html = mapper.visualize(graph, path_html=f'kepler_haxby_last02.html')

KeplerMapper()
Mapping on data shaped (1452, 464) using lens shaped (1452, 1452)

Creating 363 hypercubes.
Merged 772 duplicate nodes.

Number of nodes before merger: 2370; after merger: 1598


Created 10282 edges and 1598 nodes in 0:00:06.821292.
Wrote visualization to: kepler_haxby_last02.html


In [11]:
## Visualize the shape graph using DyNeuSR's DyNeuGraph
from dyneusr import DyNeuGraph

dG = DyNeuGraph(G=graph, y=timing_onehot)
dG.visualize('kepler_haxby_last02_dynsr.html')

label,group,value,row_count
rest,0,588,1452
scissors,1,108,1452
face,2,108,1452
cat,3,108,1452
shoe,4,108,1452
house,5,108,1452
scrambledpix,6,108,1452
bottle,7,108,1452
chair,8,108,1452


   > Found 0 nodes for data point 2.
   > Found 0 nodes for data point 3.
   > Found 0 nodes for data point 4.
   > Found 0 nodes for data point 5.
   > Found 0 nodes for data point 8.
   > Found 0 nodes for data point 31.
   > Found 0 nodes for data point 32.
   > Found 0 nodes for data point 73.
   > Found 0 nodes for data point 124.
   > Found 0 nodes for data point 240.
   > Found 0 nodes for data point 241.
   > Found 0 nodes for data point 359.
   > Found 0 nodes for data point 360.
   > Found 0 nodes for data point 361.
   > Found 0 nodes for data point 430.
   > Found 0 nodes for data point 432.
   > Found 0 nodes for data point 433.
   > Found 0 nodes for data point 434.
   > Found 0 nodes for data point 435.
   > Found 0 nodes for data point 436.
   > Found 0 nodes for data point 437.
   > Found 0 nodes for data point 485.
   > Found 0 nodes for data point 486.
   > Found 0 nodes for data point 487.
   > Found 0 nodes for data point 489.
   > Found 0 nodes for data point 490.

DyNeuGraph()